In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install ultralytics

In [ ]:
# Import necessary libraries
import torch
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # Can be commented out if not used for visualization
from ultralytics import YOLO
from pathlib import Path
import random
import csv
import os

# Set seeds (using the same seed as the original example)
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("Seeds set to 42")

In [ ]:
# Define dataset paths and class information for the 'Synthetic 2 Real 2' challenge
# Training and validation use synthetic data provided by the competition.
# Test uses the real-world images provided by the competition.
data_yaml_content = f"""
# Dataset paths (adjust if your input directory name differs slightly)
train: /kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2/train/images
val: /kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2/val/images
test: /kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2/testImages/images

# Class information
nc: 1                # number of classes
names: ['soup']      # class names (using 'soup' for the soup can)
"""

# Write the configuration to data.yaml file in the working directory
yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as file:
    file.write(data_yaml_content)

print(f"data.yaml created at {yaml_path}")
print("--- YAML Content ---")
print(data_yaml_content)
print("--------------------")

In [ ]:
# Loading pre-trained YOLO model for trainding
model = YOLO("yolov8l.pt")
print("Loaded pre-trained yolov8l.pt model.")

# Hyperparameters
print("Starting model training...")
results = model.train(
    data=yaml_path,            # Path to the data configuration file
    epochs=20,                 # Number of training epochs (adjust based on convergence)
    batch=16,                  # Batch size (adjust based on GPU memory)
    imgsz=640,                 # Input image size
    patience=20,               # Early stopping patience (stops if no improvement after N epochs)
    optimizer='SGD',           # Optimizer (SGD or Adam/AdamW are common)
    momentum=0.937,            # SGD momentum
    lr0=0.0005,                # Initial learning rate
    weight_decay=0.0001,       # Optimizer weight decay
    cos_lr=True,               # Use cosine learning rate scheduler
    save_period=-1,            # Save checkpoint frequency (-1 = only save last and best)
    workers=2,                 # Number of dataloader workers (adjust based on CPU cores)

    # Augmentations (Crucial for Sim2Real - keep or adjust based on experiments)
    hsv_h=0.015,               # Image HSV-Hue augmentation (fraction)
    hsv_s=0.7,                 # Image HSV-Saturation augmentation (fraction)
    hsv_v=0.4,                 # Image HSV-Value augmentation (fraction)
    flipud=0.5,                # Image flip up-down (probability)
    fliplr=0.5,                # Image flip left-right (probability)
    translate=0.1,             # Image translation (+/- fraction)
    scale=0.5,                 # Image scale (+/- gain)
    shear=0.01,                # Image shear (+/- deg)

    # Define project and run name (optional, for organization)
    project='S2R2_Detection',
    name='yolov8l_run'
)

print("Training finished.")
print(f"Best model weights saved to: {results.save_dir}/weights/best.pt") # Check the actual path from training output

In [ ]:
# Load the best model weights saved during training
try:
    best_model_path = Path(results.save_dir) / 'weights/best.pt'
    if not best_model_path.exists():
        print("Warning: results.save_dir not found, using default path structure.")
        best_model_path = Path('/kaggle/working/runs/detect/yolov8l_run/weights/best.pt')

    print(f"Loading best model from: {best_model_path}")
    model = YOLO(best_model_path)
except Exception as e:
    print(f"Error loading best model: {e}")
    print("Please ensure training completed successfully and the path to best.pt is correct.")

test_images_path = "/kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2/testImages/images"
output_dir = "/kaggle/working/predictions/labels"
os.makedirs(output_dir, exist_ok=True)

print(f"Running predictions on test images from: {test_images_path}")
print(f"Saving prediction TXT files to: {output_dir}")

test_image_files = list(Path(test_images_path).glob("*"))
print(f"Found {len(test_image_files)} items in test directory.")

allowed_extensions = {'.png', '.jpg', '.jpeg'}

for img_path in test_image_files:
    if not img_path.is_file() or img_path.suffix.lower() not in allowed_extensions:
        print(f"Skipping non-image file or unsupported extension: {img_path.name}")
        continue

    # conf=0.05 is a low threshold, good for competitions to maximize recall.
    try:
        results = model.predict(img_path, conf=0.05, augment=True, save_txt=False, save_conf=False) # We'll save manually

        output_txt_path = Path(output_dir) / f"{img_path.stem}.txt"

        with open(output_txt_path, "w") as f:
            if results and len(results) > 0:
                pred_result = results[0]
                img_height, img_width = pred_result.orig_shape # Get original image dimensions

                if pred_result.boxes is not None and len(pred_result.boxes) > 0:
                    for box in pred_result.boxes: # Iterate through detected boxes
                        cls_id = int(box.cls.item())
                        confidence = float(box.conf.item())
                        x1, y1, x2, y2 = box.xyxy[0].tolist()

                        x_center = ((x1 + x2) / 2) / img_width
                        y_center = ((y1 + y2) / 2) / img_height
                        width = (x2 - x1) / img_width
                        height = (y2 - y1) / img_height

                        f.write(f"0 {confidence:.6f} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    except Exception as e:
        print(f"Error predicting on image {img_path.name}: {e}")
        output_txt_path = Path(output_dir) / f"{img_path.stem}.txt"
        open(output_txt_path, 'w').close()


print(f"[notice] ✅ Predictions saved as .txt files in: {output_dir}")

In [ ]:
def predictions_to_csv(
    preds_folder: str = "/kaggle/working/predictions/labels",
    output_csv: str = "/kaggle/working/submission.csv", # Standard Kaggle submission filename
    test_images_folder: str = "/kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2/testImages/images",
    allowed_extensions: tuple = (".jpg", ".png", ".jpeg")):
    """
    Converts YOLO format prediction .txt files into a Kaggle submission CSV.

    Args:
        preds_folder: Folder containing the prediction .txt files.
        output_csv: Path to save the final submission CSV file.
        test_images_folder: Folder containing the original test images (to get all image IDs).
        allowed_extensions: Tuple of allowed image file extensions.
    """
    preds_path = Path(preds_folder)
    test_images_path = Path(test_images_folder)

    test_images_ids = {p.stem for p in test_images_path.glob("*") if p.is_file() and p.suffix.lower() in allowed_extensions}
    print(f"Found {len(test_images_ids)} unique image IDs in the test set.")

    predictions = []
    predicted_images_ids = set()

    for txt_file in preds_path.glob("*.txt"):
        image_id = txt_file.stem
        predicted_images_ids.add(image_id)

        if image_id not in test_images_ids:
             print(f"Warning: Found prediction file '{txt_file.name}' which doesn't match any known test image ID. Skipping.")
             continue

        with open(txt_file, "r") as f:
            valid_lines = [line.strip() for line in f if len(line.strip().split()) == 6]

        pred_str = " ".join(valid_lines) if valid_lines else "no boxes"
        predictions.append({"image_id": image_id, "prediction_string": pred_str})\

    missing_images_ids = test_images_ids - predicted_images_ids
    print(f"Found {len(missing_images_ids)} test images with no corresponding valid predictions (will use 'no boxes').")
    for image_id in missing_images_ids:
        predictions.append({"image_id": image_id, "prediction_string": "no boxes"})

    if not predictions:
        print("Warning: No predictions were processed. Creating an empty submission file.")
        submission_df = pd.DataFrame(list(test_images_ids), columns=['image_id'])
        submission_df['prediction_string'] = 'no boxes'
    else:
        submission_df = pd.DataFrame(predictions)


    submission_df.to_csv(output_csv, index=False, quoting=csv.QUOTE_MINIMAL)

    print(f"[notice] ✅ Submission CSV saved to {output_csv}")
    print(f"Total rows in submission file: {len(submission_df)}")
    print("Sample rows:")
    print(submission_df.head())

In [ ]:
predictions_to_csv()
print("Submission file generation process complete.")